# Polynomial Resampling

## Exploring algorithms for real-time resampling/interpolation of sensor-data.

As opposed to batchwise resampling, where you have the entire time-series with many datapoints available to recreate the signal, we are assuming that we only have a short sliding window of datapoint. This is a result of having the resampling algorithm be highly performant, and also of having to resample in real-time on an incoming stream of data.

We are also assuming that the incoming data is non-uniform in the sense that the data is not coming at specified regular interval. Some of the data might also be missing at the time when resampling has to be performed.

This document will explore some of the possibilities and options for resampling by fitting the datapoints with piecewise polynomial interpolation functions.

### Choosing the Polynomial Degree

A linear fit between datapoints is the simplest solution to implement. And one that requires the least amount of data to perform. All that is needed are two points to interpolate between. As the degree of the polynomial increases more data is required, as the polynomial has free parameters that must be determined. But, higher polynomial degree also yields better fitting curves with smoother transitions between segments of the piecewise interpolation.

We argue that cubic polynomials are the best choice as this is the smallest polynomial degree which allows one to not only align the points, but also align the first derivates resulting in smooth transisition from one polynomial piece to another.

In [97]:
import numpy as np

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models.widgets import Slider
from bokeh.layouts import row, widgetbox
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()


def modify_doc(doc):
   

    # Create the main plot
    def create_figure():
        # Set up data
        t_in = np.array([0., 1.])
        f_in = np.array([2., 3.])
        diff_in = np.array([0., 0.])

        # cubic polynomialS
        polyDeg = 3

        A = np.zeros([polyDeg + 1, polyDeg + 1])
        for i in np.arange(0, polyDeg + 1):
            A[[0, 1], i] = np.power(t_in, polyDeg - i)

        A[2, :] = np.array([3 * np.square(t_in[0]), 2 * t_in[0], 1, 0])
        A[3, :] = np.array([3 * np.square(t_in[1]), 2 * t_in[1], 1, 0])

        t = np.arange(0, 1, 0.01)

        # Get data
        f_in[0] = f1_slider.value
        f_in[1] = f2_slider.value

        diff_in[0] = diff1_slider.value
        diff_in[1] = diff2_slider.value

        # Calculate polynomial
        cubicParams = np.dot(np.linalg.inv(A), np.append(f_in, diff_in))
        y = np.polyval(cubicParams, t)
        
        # Caluclate linear diffs
        t1 = t_in[0];
        f1 = f_in[0];
        d1 = diff_in[0];
        l1 = 0.2*np.cos(np.arctan(d1))
        
        t1vec = np.arange(t1-l1,t1+l1, 0.01)
        y1vec = f1 + d1*(t1vec-t1)
        
        t2 = t_in[1];
        f2 = f_in[1];
        d2 = diff_in[1];
        l2 = 0.2*np.cos(np.arctan(d2))
        
        t2vec = np.arange(t2-l2,t2+l2, 0.01)
        y2vec = f2 + d2*(t2vec-t2)

        # Set up Plot
        plot = figure(plot_width=400, plot_height=400, x_range=[-0.3,1.3], y_range=[0.7,4.3])
        plot.line(t, y, line_width=1, color="blue")
        plot.circle(t_in, f_in, size=8, fill_color="white")
        
        plot.line(t1vec, y1vec, color="red", line_dash="dotted")
        plot.line(t2vec, y2vec, color="red", line_dash="dotted")
            
        return plot


    # Update the plot
    def update(att, old, new):
        f_in = [f1_slider.value, f2_slider.value]
        layout.children[1] = create_figure()

    # Controls
    f1_slider = Slider(start=1, end=4, value=2, step=0.01, title="f(0)")
    f2_slider = Slider(start=1, end=4, value=3, step=0.01, title="f(1)")
    diff1_slider = Slider(start=-10, end=10, value=0, step=0.01, title="f'(1)")
    diff2_slider = Slider(start=-10, end=10, value=0, step=0.01, title="f'(2)")
    
    for widget in [f1_slider, f2_slider, diff1_slider, diff2_slider]:
        widget.on_change('value', update)
    
    inputs = widgetbox(f1_slider, f2_slider, diff1_slider, diff2_slider)
    layout = row(inputs, create_figure())

    doc.add_root(layout)
    doc.title = "Polynomial Fit"
    
handler = FunctionHandler(modify_doc)
app = Application(handler)

doc = app.create_document()
show(app, notebook_url="localhost:8888")

Loading BokehJS ...

0.707106781187
